In [5]:
%run imports.ipynb

In [ ]:
def run_simulation_for_initial_condition(mae, mode, initial_condition, 
                                         exclude_degraded_state_for_average_cooperation):
    """
    Runs a single Monte Carlo simulation and returns the average cooperation and time-to-reach.

    Parameters:
        mae: The POstratAC instance (learning agent).
        information_condition_instance: The instance of Information_Conditions.
        initial_condition: The sampled initial condition for the simulation.
        initial_cooperation_in_degraded_state (int): If 0, cooperation in degraded state is set to zero;
                                                     if 1, it is set to one; otherwise, no changes.
        include_degraded_state_for_average_cooperation (bool): Whether to include the degraded state in average cooperation.

    Returns:
        tuple: (average cooperation, time-to-reach)
    """

    xtraj, fixedpointreached = mae.trajectory(initial_condition, Tmax=50000, tolerance=1e-5)

    final_point = xtraj[-1]

    avg_coop_across_states_each_agent = get_average_cooperativeness(
        policy=final_point, 
        obsdist=mae.obsdist(final_point), 
        mode=mode, 
        Oset=mae.env.Oset[0],
        exclude_degraded_state_for_average_cooperation = exclude_degraded_state_for_average_cooperation
    ) #we're only considiering agent i

    avg_coop_across_states_across_agents = np.mean(avg_coop_across_states_each_agent)

    time_to_reach = xtraj.shape[0]

    return avg_coop_across_states_across_agents, time_to_reach


In [ ]:


def run_simulation_across_conditions(mae, mode, num_samples, 
                                     exclude_degraded_state_for_average_cooperation):
    """
    Runs Monte Carlo simulations across multiple initial conditions.

    Parameters:
        mae: The POstratAC instance (learning agent).
        information_condition_instance: The instance of Information_Conditions.
        num_samples (int): Number of initial conditions to sample.
        initial_cooperation_in_degraded_state (int): If 0, cooperation in degraded state is set to zero;
                                                     if 1, it is set to one; otherwise, no changes.
        include_degraded_state_for_average_cooperation (bool): Whether to include the degraded state in average cooperation.

    Returns:
        list: A list of (average cooperation, time-to-reach) tuples.
    """
    avg_coop_time_pairs = []
    initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

    for initial_condition in initial_conditions_list:
        result = run_simulation_for_initial_condition(
            mae, mode, initial_condition,
             exclude_degraded_state_for_average_cooperation
        )
        avg_coop_time_pairs.append(result)

    return avg_coop_time_pairs


In [ ]:
def get_results_summary(avg_coop_time_pairs):
    """
    Computes summary statistics on cooperation outcomes.

    Parameters:
        avg_coop_time_pairs (list of tuples): Each tuple is (average_cooperation, time_to_reach).

    Returns:
        tuple: (average_cooperation, basin_of_attraction_size DataFrame)
    """
    df = pd.DataFrame(avg_coop_time_pairs, columns=["AverageCooperation", "TimeToReach"])
    total_count = len(df)

    # Overall average cooperation
    average_cooperation = round(df['AverageCooperation'].mean(), 3)

    # Classification
    df['Classification'] = df['AverageCooperation'].apply(
        lambda x: "Defection" if x < 0.4 else "Cooperation" if x > 0.6 else "Mixed"
    )

    # Basin of attraction statistics
    basin_of_attraction_size = df.groupby('Classification')['TimeToReach'].agg(
        MedianTimetoReach='median',
        Percentage=lambda x: round((len(x) / total_count) * 100, 1)
    ).reset_index()

    return average_cooperation, basin_of_attraction_size


In [8]:
def get_results_average_cooperation(avg_coop_time_pairs):
    """
    Computes summary statistics on cooperation outcomes.

    Parameters:
        avg_coop_time_pairs (list of tuples): Each tuple is (average_cooperation, time_to_reach).

    Returns:
        tuple: (average_cooperation, basin_of_attraction_size DataFrame)
    """
    df = pd.DataFrame(avg_coop_time_pairs, columns=["AverageCooperation", "TimeToReach"])

    # Overall average cooperation
    average_cooperation = round(df['AverageCooperation'].mean(), 3)

    return average_cooperation


In [ ]:
def get_results_only_cooperation_basin_of_attraction_size(avg_coop_time_pairs):
    """
    Computes summary statistics on cooperation outcomes.

    Parameters:
        avg_coop_time_pairs (list of tuples): Each tuple is (average_cooperation, time_to_reach).

    Returns:
        tuple: (average_cooperation, basin_of_attraction_size DataFrame)
    """
    df = pd.DataFrame(avg_coop_time_pairs, columns=["AverageCooperation", "TimeToReach"])
    total_count = len(df)

    # Overall average cooperation
    average_cooperation = round(df['AverageCooperation'].mean(), 3)

    # Classification
    df['Classification'] = df['AverageCooperation'].apply(
        lambda x: "Defection" if x < 0.4 else "Cooperation" if x > 0.6 else "Mixed"
    )
    
    basin_stats = df['Classification'].value_counts(normalize = True)*100

 
    return round(basin_stats.get('Cooperation', 0), 1)

In [ ]:


def run_simulation_and_get_results_summary(mae, mode, num_samples, 
                                     exclude_degraded_state_for_average_cooperation):

    avg_coop_time_pairs = []
    initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

    for initial_condition in initial_conditions_list:
        result = run_simulation_for_initial_condition(
            mae, mode, initial_condition,
             exclude_degraded_state_for_average_cooperation
        )
        avg_coop_time_pairs.append(result)
    results_summary = get_results_summary(avg_coop_time_pairs)
    return results_summary


In [ ]:
def make_plots_ecopgg(ecopg, mode, num_samples):

    information_condition_instance = Information_Conditions(ecopg, mode=mode)
    num_observed_states = len(information_condition_instance.Oset[0])
    x, y = ([0], list(range(num_observed_states)), [0]), ([1], list(range(num_observed_states)), [0])
    mae = POstratAC_eps(env=information_condition_instance, learning_rates=0.1, discount_factors=0.98)
    initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

    ax = fp.plot_strategy_flow(
        mae,
        x, y, flowarrow_points=np.linspace(0.01, 0.99, 9), NrRandom=32,
        conds=np.array(information_condition_instance.Oset)[0, :num_observed_states],
        # col = 'blac'
    )

    for initial_condition in initial_conditions_list:
        xtraj, fixedpointreached = mae.trajectory(initial_condition, Tmax=10000, tolerance=1e-5)
        final_point = xtraj[-1]

        fp.plot_trajectories([xtraj], x, y, cols=['grey'], lss = "--", axes = ax)

        for plot_index, (x_indices,y_indicies) in enumerate(zip(it.product(*x), it.product(*y))):
            ax[plot_index].scatter(final_point[x_indices],final_point[y_indicies], color = 'red')